# Introduction:
In this notebook we intend to use Fastai, for the purpose of image classification, our score in the competiton was ~75 %
let's see if we can improve this score.

In [ ]:
# do this to make sure no error creeps in
#! [ -e /content ] && pip install -Uqq fastai 

# basic fastai import 
from fastai.basics import *
from fastai.vision.all import *
from fastai.callback.all import *

#rest of the imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# loading data 
path = Path('../input/cassava-leaf-disease-classification')
trainFrame = pd.read_csv('/kaggle/input/cassava-leaf-disease-classification/train.csv')

Now the very first thing that we have to do is to add the path to every image name in the dataFrame

In [ ]:
trainFrame['path'] = trainFrame['image_id'].map(lambda x:path/'train_images'/x)
trainFrame = trainFrame.sample(frac=1).reset_index(drop=True)
trainFrame = trainFrame.drop(columns=['image_id'])

trainFrame[0:10]

In [ ]:
# with this done let load our data
item_tfms = RandomResizedCrop(460)
batch_tfms = [*aug_transforms(size = 224, max_warp =0), Normalize.from_stats(*imagenet_stats)]

dls = ImageDataLoaders.from_df(trainFrame, valid_pct=0.2, seed = 42, label_col = 0,
                              fn_col = 1, bs = 32, item_tfms = item_tfms,
                              batch_tfms = batch_tfms)

In [ ]:
dls.show_batch()

In [ ]:
is_pretrained = True

if is_pretrained == False:
    # let's define and train our model
    learn = cnn_learner(dls, models.resnet101, pretrained = True, metrics = [error_rate, accuracy])
    learn = learn.load('/kaggle/input/cassavafastaimodel-test/cassava_pred')
    # lets train our leaner for 5 cycles
    learn.fit_one_cycle(3)
    learn.save('cassava_pred')
else:
    learn = load_learner('/kaggle/input/cassavafastaimodel-test/cassava.pkl')

In [ ]:
sampleData = pd.read_csv(path/'sample_submission.csv')
copyFrame = sampleData.copy()
copyFrame['path'] = copyFrame['image_id'].map(lambda x:path/'test_images'/x)
copyFrame = copyFrame.drop(columns=['image_id'])

test_dl = dls.test_dl(copyFrame)
preds, _ = learn.tta(dl=test_dl, n=15, beta=0)

In [ ]:
sampleData['label'] = preds.argmax(dim=-1).numpy()
sampleData.to_csv('submission.csv',index=False)

In [ ]:
sampleData